# **Sistema de recomendación TMDB**

**Grupo_25**

**Autores:** Óscar Fernández Vázquez,
         César Regalado Núñez y
         Samuel Andrés Velásquez Semeria

Nuestro proyecto contempla la implementación de un sistema de recomendación de peliculas, con los datos recopilados de datasets formados con datos de TMDB, que es una base de datos de peliculas. Para este proyecto, se usarán varias técnicas de recuperación de datos, creación de dataframes para ver mejor los datos ya que algunos no son fáciles de visualizar, relaciones entre películas y géneros y finalmente técnicas de recomendación basadas en el contenido, en las que fundamentalmente se usarán la descripción y algunas características en común entre las películas.



# **Carga Inicial y pruebas de visualización**

Primero, dejaremos los instaladores de algunas librerias que a lo mejor no vienen por defecto en este notebook

In [217]:
!pip install surprise

In [218]:
!pip install pandas

A continuación se cargan todas las librerías que usaremos y se carga el primer dataset que usamos, el cual corresponde con el de 5000 películas de TMDB. Lo primero que haremos con este dataset es ver que información contienen sus campos.

In [219]:
# Se ejecutaran a parte para que la carga a continuación no interfiera
import pandas as pd
import ast
from ast import literal_eval
import numpy as np
from random import randint
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import SVD, Dataset, Reader

import warnings; warnings.simplefilter('ignore')


In [220]:
movies = pd.read_csv('tmdb_5000_movies.csv')

print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

# **Clasificaciones por género**

Para el primer manejo sobre los datos de este dataset clasificaremos nuestras películas.

Se hará de dos maneras distintas y que presentan distinta practicidad desde el punto de vista del que analizará los datos, o para su tratamiento posterior.

**1. Poner a que género corresponde cada película**

Para ello se creará un dataframe con las peliculas y cada columna será un género. Si la película tiene este género asociado en la columna aparece un 1 y si no lo tiene un 0.


In [221]:
# por practicidad usaremos para el ejemplo sólo 5 pelis
cabeza = movies.head() 

titles = cabeza["title"]
genres = movies["genres"]

# usamos un set, para evitar las repeticiones
generos = set()

# metemos al set los distintos generos
for i in range(len(genres)):
 salida = ast.literal_eval(genres[i])
 for j in range(len(salida)):
  generos.add(salida[j]["name"])

# la convertimos a lista para que su manejo sea más 
# facil a la hora de armar el DF
generos = list(generos) 
generos = sorted(generos) 

# creamos el nuevo dataframe con nuestras pelis como 
# filas y los géneros como columnas
df = pd.DataFrame(index=titles ,columns=generos)


Una función que nos será útil es la de rellenar las columnas con 1 o 0, el objetivo aquí es hacer esta parte del código un poco más reutilizable. El único aspecto negativo es que sólo se utilizará en los posibles casos para esta presentación de los datos

In [222]:

def rellenar_columnas(dataframe):
  # aplicamos la longitud del dataframe 
  # para que no se salga de rango
  for i in range(len(dataframe)):
    literales = ast.literal_eval(genres[i])
    generos_para_peli = set()

    for j in range(len(literales)):
      generos_para_peli.add(literales[j]['name'])
    
    generos_para_peli = list(generos_para_peli)

    for genero in generos_para_peli:
      dataframe.iloc[i][genero] = 1
      
  #poner el resto de columnas que están en NaN a 0
    for column in dataframe.columns:
      if(np.isnan(dataframe.iloc[i][column])):
        dataframe.iloc[i][column] = 0

  return dataframe

df = rellenar_columnas(df)

df # ahora podemos visualizar el dataframe resultante 


,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,
Avatar,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
Pirates of the Caribbean: At World's End,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
Spectre,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
The Dark Knight Rises,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
John Carter,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Una funcionalidad cómoda y que se puede aplicar a mayores es ver directamente la fila con la película elegida por el usuario, para ello se aplicará la siguiente función que tomará el nombre de película como entrada.

In [223]:
nombre = input("Inserte el nombre de la película(igual que en el dataframe anterior), se mostrará la fila: ")

def get_fila(nombre):
  for i in range(len(df)):
    nombre_actual = df.iloc[i].name
    if nombre_actual == nombre:
      row = pd.DataFrame(index=[nombre_actual] ,columns=generos)
      row = rellenar_columnas(row)
      return row

get_fila(nombre)



Inserte el nombre de la película(igual que en el dataframe anterior), se mostrará la fila: Avatar


,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
Avatar,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


**2. Asociar películas a generos**

Aqui se define la siguiente función, esta retornará un diccionario cuyas claves son los géneros y los valores son listas con las peliculas pertenecientes a ese género. Esta funcionalidad permite poder clasificar más facilmente a las películas por su género


In [224]:
df_generos = pd.DataFrame(index=generos)

def pelis_por_genero():
  relaciones_gen_peli = dict()

  for i in range(len(df_generos)):
    pelis_asociadas_genero = list(df_generos.iloc[i])
    genero = df_generos.iloc[i].name # me devuelve el nombre del genero
    
    for j in range(len(df)):
      genero_asociado = df.iloc[j][genero]
      if genero_asociado == 1:
        pelis_asociadas_genero.append(df.iloc[j].name)
      
    relaciones_gen_peli.update({genero:pelis_asociadas_genero})

  return relaciones_gen_peli

diccionario_generos = pelis_por_genero()

diccionario_generos # Observamos el diccionario con las asociaciones

# Nota: Habrán géneros que tendrán la lista vacía ya que de principio
# se usan muy pocas películas por practicidad en los ejemplos

{'Action': ['Avatar',
  "Pirates of the Caribbean: At World's End",
  'Spectre',
  'The Dark Knight Rises',
  'John Carter'],
 'Adventure': ['Avatar',
  "Pirates of the Caribbean: At World's End",
  'Spectre',
  'John Carter'],
 'Animation': [],
 'Comedy': [],
 'Crime': ['Spectre', 'The Dark Knight Rises'],
 'Documentary': [],
 'Drama': ['The Dark Knight Rises'],
 'Family': [],
 'Fantasy': ['Avatar', "Pirates of the Caribbean: At World's End"],
 'Foreign': [],
 'History': [],
 'Horror': [],
 'Music': [],
 'Mystery': [],
 'Romance': [],
 'Science Fiction': ['Avatar', 'John Carter'],
 'TV Movie': [],
 'Thriller': ['The Dark Knight Rises'],
 'War': [],
 'Western': []}

# **Recuperar datos de mayor relevancia desde el dataset**

Lo más común en estos sistemas es querer ver los datos asociados a la película, pero no todos, algunos de los datos que presenta el dataset sirven para hacer las operaciones de recomendación pero son irrelevantes para el usuario final. Por ello hemos decidido poner algunos de los datos más relevantes a la hora de ver una película en concreto.

In [225]:
# Estos atributos son los más básicos, se pueden ampliar si se desea

release_dates = movies["release_date"]
overviews = movies["overview"]
homepages = movies["homepage"]
languages = movies["original_language"]

columnas = ["overview", "release date", "homepage", "original language"]

df_movies = pd.DataFrame(index=titles, columns=columnas)

# aplicamos el mismo principio de antes para poner a cada peli sus datos 

for i in range(len(df_movies)):
  df_movies.iloc[i]["overview"] = overviews[i]
  df_movies.iloc[i]["release date"] = release_dates[i]
  df_movies.iloc[i]["homepage"] = homepages[i]
  df_movies.iloc[i]["original language"] = languages[i]

df_movies

,overview,release date,homepage,original language
title,,,,
Avatar,"In the 22nd century, a paraplegic Marine is di...",2009-12-10,http://www.avatarmovie.com/,en
Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",2007-05-19,http://disney.go.com/disneypictures/pirates/,en
Spectre,A cryptic message from Bond’s past sends him o...,2015-10-26,http://www.sonypictures.com/movies/spectre/,en
The Dark Knight Rises,Following the death of District Attorney Harve...,2012-07-16,http://www.thedarkknightrises.com/,en
John Carter,"John Carter is a war-weary, former military ca...",2012-03-07,http://movies.disney.com/john-carter,en


Nuevamente es conveniente hacer posible que el usuario inserte el nombre de una película y que salga la fila asociada a esta.

In [226]:
nombre = input("Inserte el nombre de la película(igual que en el dataframe anterior), se mostrará la fila: ")

def get_atributos_peli(nombre):
  for i in range(len(df_movies)):
    nombre_actual = df_movies.iloc[i].name
    if nombre_actual == nombre:
      row = pd.DataFrame(index=[nombre_actual] ,columns=columnas)
      # metemos los datos en la fila
      row.iloc[0]["overview"] = overviews[i]
      row.iloc[0]["release date"] = release_dates[i]
      row.iloc[0]["homepage"] = homepages[i]
      row.iloc[0]["original language"] = languages[i]
      return row

get_atributos_peli(nombre)

Inserte el nombre de la película(igual que en el dataframe anterior), se mostrará la fila: Avatar


,overview,release date,homepage,original language
Avatar,"In the 22nd century, a paraplegic Marine is di...",2009-12-10,http://www.avatarmovie.com/,en


Otra forma de retornar los datos asociados a una película es ponerlos en una lista (puede ser más práctico según el caso)

In [227]:
nombre = input("Inserte el nombre de la película(igual que en el dataframe anterior), se mostrará la fila: ")

def peli_info(nombre):
  pelis_con_atributos = dict()

  for i in range(len(df_movies)):
    nombre_peli = df_movies.iloc[i].name
    atributos = list(df_movies.iloc[i])
    pelis_con_atributos.update({nombre_peli:atributos})

  return pelis_con_atributos[nombre]

peli_info(nombre)

Inserte el nombre de la película(igual que en el dataframe anterior), se mostrará la fila: Avatar


['In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',
 '2009-12-10',
 'http://www.avatarmovie.com/',
 'en']

# **Implementación del sistema de recomendación**

Para este apartado utilizaremos los siguientes métodos distintos para tener un sistema bastante completo:

- Recomendaciones basadas en la descripción
- Recomendaciones por popularidad y rating
- Recomendaciones por usuario usando filtrado híbrido

**1. Recomendaciones basadas en la descripción**

Usando las descripciones se relacionarán varias películas para mostrarles al usuario. Haremos la carga de otro Dataset de películas que es un poco más completo y contiene un montón de datos importantes para nuestro sistema de recomendación.


In [228]:
mv_metadata = pd.read_csv('movies_metadata.csv')
print(mv_metadata.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Para realizar las operaciones también haremos uso de un dataset auxiliar, el cual contiene las relaciones por id de las peliculas en dos datasets distintos

In [229]:
links_small = pd.read_csv('links_small.csv')

# realizamos una conversion de tipos
# ya que los valores de interes están como string
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

mv_metadata = mv_metadata.drop([19730, 29503, 35587])

mv_metadata['id'] = mv_metadata['id'].astype('int')

# tenemos el dataframe con los datos relacionados
small_mv_metadata = mv_metadata[mv_metadata['id'].isin(links_small)]

print(small_mv_metadata.info()) # visualizar que datos tiene

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9099 entries, 0 to 45265
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9099 non-null   object 
 1   belongs_to_collection  1674 non-null   object 
 2   budget                 9099 non-null   object 
 3   genres                 9099 non-null   object 
 4   homepage               1974 non-null   object 
 5   id                     9099 non-null   int64  
 6   imdb_id                9099 non-null   object 
 7   original_language      9099 non-null   object 
 8   original_title         9099 non-null   object 
 9   overview               9087 non-null   object 
 10  popularity             9099 non-null   object 
 11  poster_path            9096 non-null   object 
 12  production_companies   9099 non-null   object 
 13  production_countries   9099 non-null   object 
 14  release_date           9099 non-null   object 
 15  rev

Para construir relaciones entre una pelicula dada usaremos los atributos 'description' y 'tagline' del dataset que hemos construido antes (small_mv_metadata). A continuación, usaremos la función para la similaridad del coseno incluida en python, la cual a través de una matriz de palabras realizará las relaciones

In [230]:
small_mv_metadata['tagline'] = small_mv_metadata['tagline'].fillna('')
small_mv_metadata['description'] = small_mv_metadata['overview'] + small_mv_metadata['tagline']
small_mv_metadata['description'] = small_mv_metadata['description'].fillna('')

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(small_mv_metadata['description'])

small_mv_metadata = small_mv_metadata.reset_index()
titles = small_mv_metadata['title']
indices = pd.Series(small_mv_metadata.index, index=small_mv_metadata['title'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

Por último, para que sea mucho más sencillo el uso se creará una funcion para obtener las recomendaciones basadas en el título pasado como argumento. Antes que nada, pondrémos un dataframe con algunos títulos de películas para que sirvan de guía.

In [231]:
small_mv_metadata[['title', 'description']].head(15)

,title,description
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...
5,Heat,"Obsessive master thief, Neil McCauley leads a ..."
6,Sabrina,An ugly duckling having undergone a remarkable...
7,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses..."
8,Sudden Death,International action superstar Jean Claude Van...
9,GoldenEye,James Bond must unmask the mysterious head of ...


Ahora la función en cuestión

In [232]:
nombre = input("Inserte el nombre de la película(igual que en las tablas anteriores), se mostrará la fila: ")

def recomendaciones_de(nombre):
    idx = indices[nombre]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

print()
recomendaciones_de(nombre).head(10) # mostramos sólo 10 para este ejemplo

Inserte el nombre de la película(igual que en las tablas anteriores), se mostrará la fila: Toy Story



2502               Toy Story 2
7535               Toy Story 3
6193    The 40 Year Old Virgin
2547           Man on the Moon
6627              Factory Girl
4702    What's Up, Tiger Lily?
889      Rebel Without a Cause
6554    For Your Consideration
4988          Rivers and Tides
1599                 Condorman
Name: title, dtype: object

**2. Recomendar películas por popularidad y ratings**

Ahora no sólo recomendará peliculas que coincidan basadas en la descripción, sino que escogerá de esa relación cual tienen un rating más alto y mayor popularidad, generando una selección filmográfica mucho más completa y de mejor calidad.

Utilizamos las clasificaciones de TMDB para crear nuestra tabla de películas principales. Usaremos la fórmula de calificación ponderada de IMDB para construir nuestro gráfico. Matemáticamente, se representa de la siguiente manera:

Calificación ponderada (WR) = (vv + m.R) + (mv + m.C)
dónde,

- **v** es el número de votos para la película

- **m** son los votos mínimos requeridos para aparecer en la tabla

- **R** es la calificación promedio de la película

- **C** es el voto medio de todo el informe

Ahora tenemos que determinar un valor apropiado para m, los votos mínimos requeridos para aparecer en la tabla. Usaremos el percentil 95 como nuestro límite. En otras palabras, para que una película aparezca en las listas, debe tener más votos que al menos el 95% de las películas de la lista.

Definimos la función en base a la fórmula anterior

In [233]:
vote_counts = mv_metadata[mv_metadata['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = mv_metadata[mv_metadata['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.95)

def calificacion_ponderada(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

A continuación la función que devolverá las películas relacionadas por rating (se recomienda usar la guía anterior para generar las entradas)

In [234]:
nombre = input("Inserte el nombre de la película(igual que en las tablas anteriores), se mostrará la fila: ")

def pelis_popularidad(nombre):
    idx = indices[nombre]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    #aqui viene el código nuevo que mejora a nuestra
    #funcion anterior
    pelis = small_mv_metadata.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date']]
    vote_counts = pelis[pelis['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = pelis[pelis['vote_average'].notnull()]['vote_average'].astype('int')

    vote_counts = pelis[pelis['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = pelis[pelis['vote_average'].notnull()]['vote_average'].astype('int')
    
    qualified = pelis[(pelis['vote_count'] >= m) & (pelis['vote_count'].notnull()) & (pelis['vote_average'].notnull())]

    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')

    qualified['wr'] = qualified.apply(calificacion_ponderada, axis=1)
    qualified = qualified.sort_values('wr', ascending=False)
    return qualified

print("\n")
pelis_popularidad(nombre).head(10) # mostramos sólo 10 para este ejemplo

# NOTA: puede que para algunas películas salgan menos de 10 recomendaciones

Inserte el nombre de la película(igual que en las tablas anteriores), se mostrará la fila: Toy Story




,title,vote_count,vote_average,release_date,wr
7535,Toy Story 3,4710,7,2010-06-16,6.851922
2502,Toy Story 2,3914,7,1999-10-30,6.824813
6290,Match Point,1134,7,2005-10-26,6.514212
994,Manhattan,600,7,1979-04-25,6.263332
6193,The 40 Year Old Virgin,2020,6,2005-08-11,5.866457
2547,Man on the Moon,444,6,1999-12-22,5.626748
4383,Maid in Manhattan,493,5,2002-12-13,5.114655


**3. Recomendaciones por usuario**

Se usarán técnicas de filtrado híbrido, que combinan el filtrado colaborativo, para tener mucha más personalización, y las recomendaciones basadas en contenido. La función aceptará como argumentos un id de usuario y una pelicula, y mediante las técnicas antes mencionadas, aunque la película sea la misma, si el usuario es diferente se recomendarán películas distintas.

Carga e inicialización de variables y datos necesarios para la parte de filtrado colaborativo

In [235]:
ratings = pd.read_csv('ratings_small.csv')
svd = SVD()
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
svd.fit(trainset)
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]

Inicialización de variables y funciones necesarias para la parte híbrida

In [236]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(small_mv_metadata[['title', 'id']], on='id').set_index('title')

indices_map = id_map.set_index('id')

Nuestra función de recomendación híbrida

In [237]:
userId = randint(1, 100) # usamos un id random en cada ejecución
nombre = input('Introduzca el nombre de la peli: ') # usaremos como ejemplo 'The Dark Knight'

def recomendacion_hibrida(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    pelis = small_mv_metadata.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date', 'id']]
    pelis['est'] = pelis['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    pelis = pelis.sort_values('est', ascending=False)
    return pelis

print("\n")
recomendacion_hibrida(userId, nombre).head(10) # usamos 10 para este ejemplo

Introduzca el nombre de la peli: The Dark Knight




,title,vote_count,vote_average,release_date,id,est
7931,The Dark Knight Rises,9263.0,7.6,2012-07-16,49026,4.336155
6144,Batman Begins,7511.0,7.5,2005-06-10,272,4.327343
2696,JFK,513.0,7.5,1991-12-20,820,4.240814
2579,Batman: Mask of the Phantasm,218.0,7.4,1993-12-25,14919,4.188591
8165,"Batman: The Dark Knight Returns, Part 1",410.0,7.7,2012-09-06,123025,4.153560
7344,Law Abiding Citizen,1522.0,7.2,2009-10-15,22803,4.047961
5511,To End All Wars,42.0,6.7,2001-09-02,1783,4.030940
7565,Batman: Under the Red Hood,459.0,7.6,2010-07-27,40662,4.026009
7933,Sherlock Holmes: A Game of Shadows,3971.0,7.0,2011-11-22,58574,4.008150
7242,The File on Thelma Jordon,8.0,6.3,1950-01-18,35404,3.966850


Para ver un comportamiento más claro y menos aleatorio, usamos una entrada fija con dos ejecuciones de código. Aunque la película sea la misma, como el usuario es distinto las recomendaciones serán ligeramente diferentes

In [238]:
# ejemplo para el usuario 1
recomendacion_hibrida(1, 'The Dark Knight').head(10)

,title,vote_count,vote_average,release_date,id,est
2579,Batman: Mask of the Phantasm,218.0,7.4,1993-12-25,14919,3.202733
6144,Batman Begins,7511.0,7.5,2005-06-10,272,3.091656
2696,JFK,513.0,7.5,1991-12-20,820,3.070894
7931,The Dark Knight Rises,9263.0,7.6,2012-07-16,49026,3.067148
8165,"Batman: The Dark Knight Returns, Part 1",410.0,7.7,2012-09-06,123025,2.898881
7933,Sherlock Holmes: A Game of Shadows,3971.0,7.0,2011-11-22,58574,2.843617
7901,Batman: Year One,255.0,7.1,2011-09-27,69735,2.794658
7344,Law Abiding Citizen,1522.0,7.2,2009-10-15,22803,2.792184
2893,Flying Tigers,7.0,6.1,1942-10-08,29372,2.786535
524,Batman,2145.0,7.0,1989-06-23,268,2.693917


In [239]:
# ejemplo para el usuario 30
recomendacion_hibrida(30, 'The Dark Knight').head(10)

,title,vote_count,vote_average,release_date,id,est
7931,The Dark Knight Rises,9263.0,7.6,2012-07-16,49026,4.114336
5511,To End All Wars,42.0,6.7,2001-09-02,1783,4.086261
8165,"Batman: The Dark Knight Returns, Part 1",410.0,7.7,2012-09-06,123025,4.085256
2696,JFK,513.0,7.5,1991-12-20,820,4.047796
6144,Batman Begins,7511.0,7.5,2005-06-10,272,3.967558
2579,Batman: Mask of the Phantasm,218.0,7.4,1993-12-25,14919,3.884157
7933,Sherlock Holmes: A Game of Shadows,3971.0,7.0,2011-11-22,58574,3.815994
7901,Batman: Year One,255.0,7.1,2011-09-27,69735,3.803287
7242,The File on Thelma Jordon,8.0,6.3,1950-01-18,35404,3.801445
6667,Fracture,908.0,7.1,2007-04-20,6145,3.769613


# **Recursos y bibliografía**

Tutoriales en github usados para aprender a manejar algunos datos: https://github.com/LearnDataSci/articles

Información de contraste sobre la similaridad del coseno: https://www.grapheverywhere.com/algoritmo-de-similitud-de-coseno/

Información de apoyo sobre el filtrado híbrido: https://medium.com/@rvillalongar/sistemas-recomendadores-hibridos-93a6fff29500